In [620]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display

# TODOs:
# automate tracefile generation and symbol analysis
# get an error bound based on the data
# analyze trace with full symbols list
# filter out useless symbol (or just bar chart/visualize everything)
# Get CONSISTENT data

att = pd.read_csv("data/attestation_1_full.csv") # correct data
# att2 = pd.read_csv("data/attestation2.csv") # bad data
# attfull = pd.read_csv("data/attestationfull.csv")

# aes = pd.read_csv("data/tinyaes.csv")
# aes2 = pd.read_csv("data/tinyaes2.csv")
# aesfull = pd.read_csv("data/tinyaesfull.csv")

In [621]:
class Boundary:
    """A boundary is defined by a start and end symbol along with a name."""
    def __init__(self, name, start, end):
        self.name = name
        self.start = start
        self.end = end

In [622]:
"""Given a set of boundaries (in no order) gather the number of cycles it took
to reach the end of each of the boundaries, by returning a pandas dataframe of
the boundaries, their labels, and the cycle cost. This should work in the case
of multiple enclaves being started, running a user/runtime, and then destroying.
"""

def get_indicies(bset, data):
    """Return the indicies of all the found possible boundaries.
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    indicies = []
    for i, row in data.iterrows():
        if row['symbol'] in start_symbols or row['symbol'] in end_symbols:
            if i not in indicies:
                indicies.append(i)
                
    return indicies

def get_cost_table(data):
    """Return table with cycle cost in place of the relative cycle number
    """
    data["end"] = data["symbol"].shift(-1)
    data["cost"] = -(data['cycle'] - data['cycle'].shift(-1))
    # drop the last row because this is cycle cost per instruction
    data = data.drop(['cycle'], axis=1)[:-1]
    return data

def get_symbol_cost_table(phase_df, data):
    """Given a phase cycles table, extract the start and end indicies for each phase, and break
    down those symbols into a pivot table from the original data table.
    """
    # get cost table from original data
    cost_data = get_cost_table(data)
    result = []
    for _, row in phase_df.iterrows():
        start = row['start_index']
        phase = row['phases']
        end = row['end_index']
        symbol_data = cost_data[start:end].pivot(columns='symbol', values='cost').bfill().iloc[[0],:]
        symbol_data['phase'] = phase
        # display(symbol_data)
        result.append(symbol_data)
    # merge dataframes and index on phase
    final = pd.concat(result, ignore_index=True).set_index('phase')
    # sort columns by mean value within column
    return final.reindex(final.mean().sort_values().index, axis=1)

def get_phase_cycles_table(indicies, bset, data):
    """Given indicies of interest and the corresponding data
    return the cycle count of each phase, along with the 
    start and end indicies of each phase which can be used
    for further analysis
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    
    # focus the data
    df = data.iloc[indicies]
    # boundary matching
    phases = []
    cycle_counts = []
    start_indicies = []
    end_indicies = []
    for startidx, row in df.iterrows():
        # continue up until the startidx equals the previous endidx
        if len(end_indicies) > 0:
            if startidx < end_indicies[-1]:
                continue
        start = row['symbol']
        # scan everything in front of the current row
        for endidx, next_row in df.loc[startidx:].iterrows():
            end = next_row['symbol']
            # print(start, end)
            found_end = False
            for b in bset:
                if b.start == start and b.end == end:
                    total_cycles = next_row['cycle'] - row['cycle'] 
                    phases.append(b.name)
                    cycle_counts.append(total_cycles)
                    start_indicies.append(startidx)
                    end_indicies.append(endidx)
                    found_end = True
                    break # on first one, we can stop
            if found_end:
                break
    
    labels = ["phases", "cycle_counts", "start_index", "end_index"]
    res = pd.DataFrame([phases, cycle_counts, start_indicies, end_indicies]).transpose()
    res.columns = labels
    return res


# Macro boundaries for the enclave
# We will also want to have smaller microboundaries for the enclave within these larger bucket.
# For these, we will not create separate boundaries.
# With the boundaries within boundaries, we can identify what's slowing down the enclave.
# Smaller bounds should come first.
# create_bound = Boundary("create_enclave", "keystone_create_enclave", "keystone_run_enclave")
# run_bound = Boundary("run_enclave", "keystone_run_enclave", "rt_base")
# runtime_attest_bound = Boundary("enclave_runtimeuser4", "rt_base", "mcall_sm_not_implemented") # weird case
# runtime_bound = Boundary("enclave_runtimeuser", "rt_base", "mcall_sm_attest_enclave")
# attest_copy_bound = Boundary("enclave_attestation2", "mcall_sm_not_implemented", "return_to_encl") # weird case
# attest_bound = Boundary("enclave_attestation", "mcall_sm_attest_enclave", "return_to_encl")
# return_to_runtime_bound = Boundary("enclave_runtimeuser1", "return_to_encl", "mcall_sm_stop_enclave")
# attest_edge_call_bound2 = Boundary("enclave_attestation_edge_call", "dispatch_edgecall_ocall", "return_to_encl")
# attest_edge_call_bound = Boundary("enclave_attestation_edge_call", "mcall_sm_stop_enclave", "return_to_encl")
# run_to_exit_bound = Boundary("enclave_runtimeuser_until_ocall", "return_to_encl", "dispatch_edgecall_ocall")
# run_to_exit_bound = Boundary("enclave_runtimeuser2", "return_to_encl", "mcall_sm_exit_enclave")
# run_wout_attest_bound = Boundary("enclave_runtimeuser3", "rt_base","mcall_sm_stop_enclave")
# destroy_bound = Boundary("destroy_enclave", "destroy_enclave", "keystone_destroy_enclave_end")
# destroy_bound_full = Boundary("destroy_enclave", "destroy_enclave", "_raw_spin_unlock_bh")

# enclave_bset = [create_bound, run_bound, runtime_attest_bound, runtime_bound, attest_copy_bound, attest_bound, return_to_runtime_bound, 
                # attest_edge_call_bound2, attest_edge_call_bound, run_to_exit_bound, run_wout_attest_bound, destroy_bound, destroy_bound_full]

native_bset = [] # we just want user mode?
ignore_set = ["printf"]

In [623]:
# Smaller bounds should come first.
create = Boundary("create_enclave", "keystone_create_enclave", "keystone_run_enclave")
run_overhead = Boundary("run_enclave", "keystone_run_enclave", "rt_base")

runtime = Boundary("enclave_runtimeuser", "rt_base", "mcall_sm_attest_enclave")
runtime2 = Boundary("enclave_runtimeuser2", "return_to_encl", "mcall_sm_stop_enclave")
runtime3 = Boundary("enclave_runtimeuser3", "return_to_encl", "mcall_sm_exit_enclave")

attest = Boundary("enclave_attestation", "mcall_sm_attest_enclave", "return_to_encl")
attest_ecall = Boundary("enclave_attestation_edge_call", "mcall_sm_stop_enclave", "return_to_encl")

runtime_noattest = Boundary("enclave_runtimeuser", "rt_base","mcall_sm_stop_enclave")
exit_enclave = Boundary("exit_enclave", "mcall_sm_exit_enclave", "keystone_ioctl")
destroy_bound = Boundary("destroy_enclave", "destroy_enclave", "keystone_destroy_enclave_end")


enclave_bset = [create, run_overhead, runtime, runtime2, runtime3, attest, attest_ecall, runtime_noattest,
               exit_enclave, destroy_bound]

In [624]:
# Run Analysis
def get_large_buckets(data):
    bset = enclave_bset
    indicies = get_indices(bset, data)
    display(data.iloc[indicies])
    phase_table = get_phase_cycles_table(indicies, bset, data)
    display(phase_table)
    results = get_symbol_cost_table(phase_table, data)
    return results

In [625]:
att = get_large_buckets(att)
display(att)
# display(aes[:50])
# aes2res = get_large_buckets(aes2)
# display(aes2res)
# attres = get_large_buckets(att)
# display(attres)
# att2res = get_large_buckets(att2) # looks odd.
# display(att2res)

,cycle,env,symbol
0,4591681899,linux,keystone_create_enclave
4722,4641235890,linux,keystone_ioctl
4725,4641236557,linux,keystone_run_enclave
4746,4641241402,runtime,rt_base
4784,4641245522,sm,mcall_sm_attest_enclave
8937,4642520390,runtime,return_to_encl
8968,4642527293,sm,mcall_sm_stop_enclave
16107,4647438248,linux,keystone_ioctl
16131,4647444819,runtime,return_to_encl
16144,4647446906,sm,mcall_sm_exit_enclave


,phases,cycle_counts,start_index,end_index
0,create_enclave,49554658,0,4725
1,run_enclave,4845,4725,4746
2,enclave_runtimeuser,4120,4746,4784
3,enclave_attestation,1274868,4784,8937
4,enclave_runtimeuser2,6903,8937,8968
5,enclave_attestation_edge_call,4917526,8968,16131
6,enclave_runtimeuser3,2087,16131,16144
7,exit_enclave,82493,16144,16440


,hash_extend,hash_finalize,hash_extend_page,work_pending,osm_pmp_set,ge_p3_0,get_enclave_by_id,alloc_request_simple,blk_delay_work,tlb_flush_mmu,...,__find_get_block,ext2_get_branch,_mix_pool_bytes,fe_mul,unmapped_area,sc_reduce,sc_muladd,restore_all,memset,sha3_keccakf
phase,,,,,,,,,,,,,,,,,,,,,
create_enclave,2.0,2.0,3.0,5.0,NaN,NaN,NaN,10.0,12.0,NaN,...,2131.0,2333.0,2417.0,NaN,3175.0,NaN,NaN,791.0,193346.0,77945.0
run_enclave,NaN,NaN,NaN,NaN,6.0,NaN,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enclave_runtimeuser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enclave_attestation,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2705.0,NaN,3975.0,7816.0,NaN,NaN,77831.0
enclave_runtimeuser2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
enclave_attestation_edge_call,NaN,NaN,NaN,NaN,6.0,NaN,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24379.0,432.0,NaN
enclave_runtimeuser3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
exit_enclave,NaN,NaN,NaN,4.0,6.0,NaN,NaN,NaN,NaN,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11770.0,414.0,NaN


In [626]:
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

import cufflinks as cf

# You'll need to set API key using:
# plotly.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')
# Find your api_key here: https://plot.ly/settings/api
# Cufflinks binds plotly to pandas dataframes in IPython notebook.

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

att_sorted.iplot(kind='bar', barmode="stack", filename='name.html')